In [1]:
import numpy as np

import matplotlib.pyplot as plt

import sys
sys.path.insert(0,"C:/Users/Dustin/Desktop/Numerische Methoden/VS Code/NM-Code-for-Excercises")
import shortcuts as s

In [2]:
#x = 3

def f(x):
    return (x**4)*(np.log10(x+(1+x**2)**0.5))          #passt, habs gecheckt, f ist ein numpyfloat, ich gehe davon aus, dass mit log, log10 gemeint ist

In [3]:
#Extended trapezoidal rule

def integral_ext_trap(f,a,b,N):                     #funktioniert
    if b >= a:
        h = (b-a)/N
        x = a
        area = 0.5*(f(a)+f(b))
        i = 1
        while i < N:
            area += f(x+i*h)
            i += 1
        area = h*area
    else:
        h = (a-b)/N
        x = b
        area = 0.5*(f(a)+f(b))
        i = 1
        while i <= N:
            area += f(i)
            i += 1
        area = -h*area
    return area

In [10]:
I = s.integral_ext_trap(f,0,2,100000)

exact = 3.5409610461819527733248015434794460

error = (exact-I)/exact

print(I)
print(error)

3.5409610469542954
-2.1811667064929294e-10


In [5]:
#Extended Simpson-Rule

def integral_ext_simp(f,a,b,N):                         #funktioniert
    if b >= a:
        h = (b-a)/N
        x = a
        area = (1/3)*(f(a)+f(b))
        i = 1
        while i < N:
            if (i % 2 == 0):
                area += (2/3)*f(x+i*h)
            else:
                area += (4/3)*f(x+i*h)
            i += 1
        area = h*area
    else:
        h = (a-b)/N
        x = b
        area = (1/3)*(f(a)+f(b))
        i = 1
        while i < N:
            if (i % 2 == 0):
                area += (2/3)*f(x+i*h)
            else:
                area += (4/3)*f(x+i*h)
            i += 1
        area = h*area
    return area

In [8]:
I = s.integral_ext_simp(f,0,2,1000)

exact = 3.5409610461819527733248015434794460

error = (exact-I)/exact

print(I)
print(error)

3.5409610461865273
-1.2918986938865157e-12


In [7]:
#Romberg integration

def integral_romberg(f,a,b,N):
    if b >= a:
        h = (b-a)/(2**N)
        
    else:
        h = (a-b)/(2**N)
        
    return area
    